In [7]:
pip install mido

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade git+https:// github.com/louisabraham/python3-midi

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'github.com/louisabraham/python3-midi'
Hint: It looks like a path. File 'github.com/louisabraham/python3-midi' does not exist.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import os
import midi
import numpy as np

from mido import MidiFile, MidiTrack, Message

In [9]:
num_notes = 96
samples_per_measure = 96

def midi_to_samples(fname):
	has_time_sig = False
	flag_warning = False
	mid = MidiFile(fname)
	ticks_per_beat = mid.ticks_per_beat
	ticks_per_measure = 4 * ticks_per_beat

	for i, track in enumerate(mid.tracks):
		for msg in track:
			if msg.type == 'time_signature':
				new_tpm = msg.numerator * ticks_per_beat * 4 / msg.denominator
				if has_time_sig and new_tpm != ticks_per_measure:
					flag_warning = True
				ticks_per_measure = new_tpm
				has_time_sig = True
	if flag_warning:
		print ("  ^^^^^^ WARNING ^^^^^^ " )
		return []
	
	all_notes = {}
	for i, track in enumerate(mid.tracks):
		abs_time = 0
		for msg in track:
			abs_time += msg.time
			if msg.type == 'note_on':
				if msg.velocity == 0:
					continue
				note = msg.note - (128 - num_notes)/2
				assert(note >= 0 and note < num_notes)
				if note not in all_notes:
					all_notes[note] = []
				else:
					single_note = all_notes[note][-1]
					if len(single_note) == 1:
						single_note.append(single_note[0] + 1)
				all_notes[note].append([abs_time * samples_per_measure / ticks_per_measure])
			elif msg.type == 'note_off':
				if len(all_notes[note][-1]) != 1:
					continue
				all_notes[note][-1].append(abs_time * samples_per_measure / ticks_per_measure)
	for note in all_notes:
		for start_end in all_notes[note]:
			if len(start_end) == 1:
				start_end.append(start_end[0] + 1)
	samples = []
	for note in all_notes:
		for start, end in all_notes[note]:
			sample_ix = start / samples_per_measure
			while len(samples) <= sample_ix:
				samples.append(np.zeros((samples_per_measure, num_notes), dtype=np.uint8))
			sample = samples[sample_ix]
			start_ix = start - sample_ix * samples_per_measure
			if False:
				end_ix = min(end - sample_ix * samples_per_measure, samples_per_measure)
				while start_ix < end_ix:
					sample[start_ix, note] = 1
					start_ix += 1
			else:
				sample[start_ix, note] = 1
	return samples

def samples_to_midi(samples, fname, ticks_per_sample, thresh=0.5):
	mid = MidiFile()
	track = MidiTrack()
	mid.tracks.append(track)
	ticks_per_beat = mid.ticks_per_beat
	ticks_per_measure = 4 * ticks_per_beat
	ticks_per_sample = ticks_per_measure / samples_per_measure
	abs_time = 0
	last_time = 0
	for sample in samples:
		for y in xrange(sample.shape[0]):
			abs_time += ticks_per_sample
			for x in xrange(sample.shape[1]):
				note = x + (128 - num_notes)/2
				if sample[y,x] >= thresh and (y == 0 or sample[y-1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_on', note=note, velocity=127, time=delta_time))
					last_time = abs_time
				if sample[y,x] >= thresh and (y == sample.shape[0]-1 or sample[y+1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_off', note=note, velocity=127, time=delta_time))
					last_time = abs_time
	mid.save(fname)


In [ ]:
## LOAD SONGS ##
patterns = {}
dirs = ["data/"]
all_samples = []
all_lens = []
print ("Loading Songs...")
for dir in dirs:
    for root, subdirs, files in os.walk(dir):
        for file in files:
            path = root + "/" + file
            if not (path.endswith('.mid') or path.endswith('.midi')):
                continue
            try:
                samples = midi_to_samples(path)
            except Exception as inst:
                print ("ERROR ", path, inst)
                continue
            if len(samples) < 8:
                continue

            samples, lens = util.generate_add_centered_transpose(samples)
            all_samples += samples
            all_lens += lens
assert(sum(all_lens) == len(all_samples))
print ("Saving " + str(len(all_samples)) + " samples...")
all_samples = np.array(all_samples, dtype=np.uint8)
all_lens = np.array(all_lens, dtype=np.uint32)
np.save('samples.npy', all_samples)
np.save('lengths.npy', all_lens)
print ("Done")

Loading Songs...
ERROR  data/Midi/2a60530c7dccc812f5b8acaed9ade9ed.mid list indices must be integers or slices, not float
ERROR  data/Midi/c1570bd149d3b8e63e45be58552fea19.mid list indices must be integers or slices, not float
ERROR  data/Midi/0d15ccb09ec9efa67f933ec37c56c5eb.mid list indices must be integers or slices, not float
ERROR  data/Midi/3268d19b81145078d1a8f450feb07d7b.mid 
ERROR  data/Midi/a478302cafde2aa91b28e57cae9efea7.mid 
ERROR  data/Midi/7daacb647e8968e7422d59912f8153fd.mid list indices must be integers or slices, not float
ERROR  data/Midi/3abdac41f329dda66f7963d47bb311aa.mid list indices must be integers or slices, not float
ERROR  data/Midi/b3873990c90cfd2a6fe3ae8040ecb0af.mid list indices must be integers or slices, not float
ERROR  data/Midi/56699e4a5f6343f56f0179ebfa6fc078.mid list indices must be integers or slices, not float
ERROR  data/Midi/b8e21a6a26ea481e0301463cd6311bf4.mid list indices must be integers or slices, not float
ERROR  data/Midi/587bf1c0c607b5b1

ERROR  data/Midi/e09fdfa458393588814ae8b79a6367a1.mid list indices must be integers or slices, not float
ERROR  data/Midi/1a03f8a25acd4bdea577cfa54642a4c1.mid list indices must be integers or slices, not float
ERROR  data/Midi/fd81d4448012eb2b64e5e8fc81a5f963.mid list indices must be integers or slices, not float
  ^^^^^^ WARNING ^^^^^^ 
ERROR  data/Midi/1fa72c6a86c91a7af9e98916f5b26cb1.mid list indices must be integers or slices, not float
ERROR  data/Midi/6e6e0d0e0ccdf7116f3fb27a3efa97f3.mid list indices must be integers or slices, not float
ERROR  data/Midi/989c9875fc96941af527f497d7380f22.mid list indices must be integers or slices, not float
ERROR  data/Midi/93f68ccd598057a537cfc59f4004578b.mid list indices must be integers or slices, not float
ERROR  data/Midi/379d20a5b65c96c0dd6bcf851d9f44b1.mid list indices must be integers or slices, not float
ERROR  data/Midi/a599f1b4ba63a967c920c223536b5e92.mid list indices must be integers or slices, not float
ERROR  data/Midi/657551c367279

ERROR  data/Midi/6e085eb45d0478d2fe581079ef6140a3.mid list indices must be integers or slices, not float
ERROR  data/Midi/83777120274dbee46baaf6efceaf9ce3.mid list indices must be integers or slices, not float
ERROR  data/Midi/a2883b6a8a85f7d2e82f614e9878acc2.mid list indices must be integers or slices, not float
ERROR  data/Midi/1ca6d88d918bcea57d950e4b10a5eb74.mid list indices must be integers or slices, not float
ERROR  data/Midi/03de0ec82c000a7a6a4e0f08b5c47baa.mid list indices must be integers or slices, not float
ERROR  data/Midi/09d3ebe900c43f2281a9cc10869b56a7.mid list indices must be integers or slices, not float
ERROR  data/Midi/7050706e2bf7f96c3c9b7526000a785d.mid list indices must be integers or slices, not float
ERROR  data/Midi/102bb99ef51ff099abecbcb0695aaae5.mid list indices must be integers or slices, not float
ERROR  data/Midi/e2efe8187edd18c366830190fa0dd778.mid list indices must be integers or slices, not float
ERROR  data/Midi/29f1e597771e9acdc1661ec1a191594c.mid l

  ^^^^^^ WARNING ^^^^^^ 
ERROR  data/Midi/1412c571610c60688546506a4a660f8a.mid list indices must be integers or slices, not float
ERROR  data/Midi/63de8e99e2be453b84524cfea68e64aa.mid list indices must be integers or slices, not float
ERROR  data/Midi/d8212e4cb6fb761c464b2251abc2e30f.mid list indices must be integers or slices, not float
ERROR  data/Midi/90ccfaca980452ad67b5ebefc78dbf49.mid list indices must be integers or slices, not float
ERROR  data/Midi/2c6815881ce80da58374a9e462cacaac.mid list indices must be integers or slices, not float
  ^^^^^^ WARNING ^^^^^^ 
ERROR  data/Midi/0e320f1d4fa1383e6673db1554e7b76e.mid list indices must be integers or slices, not float
ERROR  data/Midi/5e4077ffdcc9b67b9e4507d92490eba6.mid list indices must be integers or slices, not float
ERROR  data/Midi/3dc74cd779f3886360d3937fbbbf6e16.mid 
ERROR  data/Midi/5fdec372b630ce6fe62e9081c8a6d43e.mid list indices must be integers or slices, not float
ERROR  data/Midi/9d6b939dd0eae2ffdde8dd83fd12eecb.mid l

ERROR  data/Midi/afd31bf36e75b8ecaa79dbdcea947ede.mid list indices must be integers or slices, not float
ERROR  data/Midi/54363050aa76c3e6494ab8ffe47e8a43.mid list indices must be integers or slices, not float
ERROR  data/Midi/561d3c65f72ffb24ae25de2867c5dfea.mid list indices must be integers or slices, not float
ERROR  data/Midi/8e11c74c97ab39164e697240b17de89c.mid list indices must be integers or slices, not float
ERROR  data/Midi/f19e6ad0674207bfadc5839dca5c89af.mid list indices must be integers or slices, not float
ERROR  data/Midi/14cbceaac78259096c60b7a1a67ad4aa.mid list indices must be integers or slices, not float
ERROR  data/Midi/c92bb6a400ccbaa830b0ed9fa789a46e.mid list indices must be integers or slices, not float
ERROR  data/Midi/bd1007f4a2a247c8864211d67c2f0bf5.mid list indices must be integers or slices, not float
ERROR  data/Midi/1ee6696761eb0c1826ca96bf5c9c595b.mid list indices must be integers or slices, not float
ERROR  data/Midi/21ed504be5be365e8e1bf9909d087cfa.mid l

...

ERROR  data/Midi/a88ee5e30d340c8feb290298ba66b672.mid list indices must be integers or slices, not float
ERROR  data/Midi/feb0a94081c71245c0776044056e8fb0.mid list indices must be integers or slices, not float
ERROR  data/Midi/01e7a692604e78cad1cc4db21ab92723.mid 
ERROR  data/Midi/1cbe4e75332d8cd775eb69ba06cbad44.mid list indices must be integers or slices, not float
ERROR  data/Midi/92ce77be471b0f97fc0d23ed0b41070c.mid list indices must be integers or slices, not float
ERROR  data/Midi/d76ec01d2f53de5991391243201568d6.mid list indices must be integers or slices, not float
ERROR  data/Midi/9f96135bcc87cb059bf13cfd0847d3fe.mid list indices must be integers or slices, not float
ERROR  data/Midi/338be63c15c45ff9a45b433336aa7221.mid list indices must be integers or slices, not float
ERROR  data/Midi/545abbfee00eb284c51cdb01da60dc64.mid list indices must be integers or slices, not float
ERROR  data/Midi/7358612b7d186c818139be39e6d0ca7f.mid list indices must be integers or slices, not float


ERROR  data/Midi/eb504f29b1a10567814f198e7e049d15.mid list indices must be integers or slices, not float
ERROR  data/Midi/783e6bd3cdde1e33a7890a161de7e59a.mid list indices must be integers or slices, not float
ERROR  data/Midi/8d11714f7ff446de96549fd9d31fcedb.mid list indices must be integers or slices, not float
ERROR  data/Midi/078e7e00b55c8662f3275d4c025245f1.mid list indices must be integers or slices, not float
ERROR  data/Midi/c5d7fb46015327c1f16b22f2a650df08.mid list indices must be integers or slices, not float
  ^^^^^^ WARNING ^^^^^^ 
ERROR  data/Midi/3d943616830150eef48eb0f77a8628ee.mid list indices must be integers or slices, not float
ERROR  data/Midi/9c6224227335359fda05e52cadcbb528.mid 
ERROR  data/Midi/b59eea3595b35d65c2ae67f48825765a.mid list indices must be integers or slices, not float
ERROR  data/Midi/ed168a7e80ceca46c57b39a5aab7d8f4.mid list indices must be integers or slices, not float
ERROR  data/Midi/052d7233a51efce2094558f61fb563e1.mid list indices must be integ

ERROR  data/Midi/ba5f7b8a8dbb63339c6e882e27440154.mid list indices must be integers or slices, not float
ERROR  data/Midi/766e14848d4c5fc4a80ae2b99edf967c.mid list indices must be integers or slices, not float
ERROR  data/Midi/271c809c20b07cd27197e7f687856c0d.mid list indices must be integers or slices, not float
ERROR  data/Midi/f31217fa561d7ffdf2418c17bc419017.mid list indices must be integers or slices, not float
ERROR  data/Midi/cd65b176f1379b5ccf116f00657347f1.mid list indices must be integers or slices, not float
ERROR  data/Midi/09f471409da7ae640fb4ec5264c991ba.mid list indices must be integers or slices, not float
ERROR  data/Midi/678ab79f5bd6d6beb256c56ff8716eb4.mid list indices must be integers or slices, not float
  ^^^^^^ WARNING ^^^^^^ 
ERROR  data/Midi/97c67f4138b1f71d828ce4117b203c7c.mid list indices must be integers or slices, not float
ERROR  data/Midi/0e1b4ec3892e9b807ea866f7309d5453.mid list indices must be integers or slices, not float
ERROR  data/Midi/334e2af1af4c5

ERROR  data/Midi/e2f9f09735e7ffc8d977f2daa55117d6.mid list indices must be integers or slices, not float
ERROR  data/Midi/690b3553bac57aa58a723cb176105a3d.mid list indices must be integers or slices, not float
ERROR  data/Midi/625b6adf5450e7e3fcdf86fb8a8d6c04.mid list indices must be integers or slices, not float
ERROR  data/Midi/a305292beabb03fa5630f867cab946ef.mid list indices must be integers or slices, not float
ERROR  data/Midi/728c3dbdc9b47142dc8f725c6805c259.mid list indices must be integers or slices, not float
ERROR  data/Midi/b0493abb18a9b7da99cf3ac222b2c41d.mid list indices must be integers or slices, not float
ERROR  data/Midi/921259a09728e5a0bcef7e63b3e9a3f9.mid list indices must be integers or slices, not float
ERROR  data/Midi/f97de7b0cadb01f51f750ada2f91a170.mid list indices must be integers or slices, not float
ERROR  data/Midi/c336e46a8cabefae8c89480431cbe8be.mid list indices must be integers or slices, not float
ERROR  data/Midi/05604bc6466b4cbc4c2dd747f1c8ebd4.mid l

ERROR  data/Midi/beea331af3631542778128dca2139be3.mid list indices must be integers or slices, not float
ERROR  data/Midi/dd53c4a072288336c45710e1699aa4a6.mid list indices must be integers or slices, not float
ERROR  data/Midi/ffd19b7f18f18caeabf8ea55363e9fb6.mid list indices must be integers or slices, not float
ERROR  data/Midi/ec9d14ec92f7ce6857105ed979595e73.mid list indices must be integers or slices, not float
ERROR  data/Midi/b0ba3340622c50edaefda1b31cedc269.mid 
ERROR  data/Midi/04dbccf9a588a591faa35cceb3d30f99.mid list indices must be integers or slices, not float
ERROR  data/Midi/7490c7a524050bca0f5759241446352c.mid list indices must be integers or slices, not float
ERROR  data/Midi/520deff71228d5bd93460c88b223ac6f.mid list indices must be integers or slices, not float
ERROR  data/Midi/37cb5f99eee81b81ec2276755f0845b3.mid list indices must be integers or slices, not float
ERROR  data/Midi/03dc6761a6861d1758585cb496450d0d.mid list indices must be integers or slices, not float


In [11]:
## utility functions ##

def transpose_range(samples):
	merged_sample = np.zeros_like(samples[0])
	for sample in samples:
		merged_sample = np.maximum(merged_sample, sample)
	merged_sample = np.amax(merged_sample, axis=0)
	min_note = np.argmax(merged_sample)
	max_note = merged_sample.shape[0] - np.argmax(merged_sample[::-1])
	return min_note, max_note

def generate_add_centered_transpose(samples):
	num_notes = samples[0].shape[1]
	min_note, max_note = transpose_range(samples)
	s = num_notes/2 - (max_note + min_note)/2
	out_samples = samples
	out_lens = [len(samples), len(samples)]
	for i in xrange(len(samples)):
		out_sample = np.zeros_like(samples[i])
		out_sample[:,min_note+s:max_note+s] = samples[i][:,min_note:max_note]
		out_samples.append(out_sample)
	return out_samples, out_lens
	
def generate_all_transpose(samples, radius=6):
	num_notes = samples[0].shape[1]
	min_note, max_note = transpose_range(samples)
	min_shift = -min(radius, min_note)
	max_shift = min(radius, num_notes - max_note)
	out_samples = []
	out_lens = []
	for s in xrange(min_shift, max_shift):
		for i in xrange(len(samples)):
			out_sample = np.zeros_like(samples[i])
			out_sample[:,min_note+s:max_note+s] = samples[i][:,min_note:max_note]
			out_samples.append(out_sample)
		out_lens.append(len(samples))
	return out_samples, out_lens

def sample_to_pic(fname, sample, thresh=None):
	if thresh is not None:
		inverted = np.where(sample > thresh, 0, 1)
	else:
		inverted = 1.0 - sample
	cv2.imwrite(fname, inverted * 255)

def samples_to_pics(dir, samples, thresh=None):
	if not os.path.exists(dir): os.makedirs(dir)
	for i in xrange(samples.shape[0]):
		sample_to_pic(dir + '/s' + str(i) + '.png', samples[i], thresh)

def pad_songs(y, y_lens, max_len):
	y_shape = (y_lens.shape[0], max_len) + y.shape[1:]
	y_train = np.zeros(y_shape, dtype=np.float32)
	cur_ix = 0
	for i in xrange(y_lens.shape[0]):
		end_ix = cur_ix + y_lens[i]
		for j in xrange(max_len):
			k = j % (end_ix - cur_ix)
			y_train[i,j] = y[cur_ix + k]
		cur_ix = end_ix
	assert(end_ix == y.shape[0])
	return y_train

def sample_to_pattern(sample, ix, size):
	num_pats = 0
	pat_types = {}
	pat_list = []
	num_samples = len(sample) if type(sample) is list else sample.shape[0]
	for i in xrange(size):
		j = (ix + i) % num_samples
		measure = sample[j].tobytes()
		if measure not in pat_types:
			pat_types[measure] = num_pats
			num_pats += 1
		pat_list.append(pat_types[measure])
	return str(pat_list), pat_types

def embed_samples(samples):
	note_dict = {}
	n, m, p = samples.shape
	samples.flags.writeable = False
	e_samples = np.empty(samples.shape[:2], dtype=np.int32)
	for i in xrange(n):
		for j in xrange(m):
			note = samples[i,j].data
			if note not in note_dict:
				note_dict[note] = len(note_dict)
			e_samples[i,j] = note_dict[note]
	samples.flags.writeable = True
	lookup = np.empty((len(note_dict), p), dtype=np.float32)
	for k in note_dict:
		lookup[note_dict[k]] = k
	return e_samples, note_dict, lookup

def e_to_samples(e_samples, lookup):
	samples = np.empty(e_samples.shape + lookup.shape[-1:], dtype=np.float32)
	n, m = e_samples.shape
	for i in xrange(n):
		for j in xrange(m):
			samples[i,j] = lookup[e_samples[i,j]]
	return samples

In [ ]:
ls